In [49]:
import torchvision
import torch
import sys

sys.path.append("..")

from torch.utils.data import DataLoader, Dataset
from utils.model_util import LeNet5, CNN, MLP
from utils.train_util import *

In [61]:
model = LeNet5(28, 28, 1, 10).cuda()
train_dataset_loader = DataLoader(
    dataset=torchvision.datasets.MNIST(
        root='../data/raw-data/',
        train=True,
        transform=torchvision.transforms.ToTensor()),
    batch_size=600,
    shuffle=False)
test_dataset_loader = DataLoader(
    dataset=torchvision.datasets.mnist.MNIST(
        root='../data/raw-data/',
        transform=torchvision.transforms.ToTensor()),
    batch_size=32,
    shuffle=False)
test_dataset = torchvision.datasets.mnist.MNIST(
    root='../data/raw-data/',
    transform=torchvision.transforms.ToTensor())

opti = torch.optim.Adam(model.parameters())
criterion = torch.nn.CrossEntropyLoss(reduction='sum')
for data, target in train_dataset_loader:
    opti.zero_grad()
    output = model(data.cuda())
    loss = criterion(output, target.cuda())
    loss.backward()
    print(loss.item())
    opti.step()

print(loss, type(loss))
print(loss.item(), type(loss.item()))

1380.2177734375
1383.3494873046875
1380.1983642578125
1379.912109375
1377.591552734375
1374.9847412109375
1372.6951904296875
1372.326171875
1368.70458984375
1366.3475341796875
1360.7509765625
1360.9248046875
1355.194580078125
1345.769287109375
1335.60205078125
1330.85498046875
1316.6502685546875
1302.5662841796875
1288.5211181640625
1280.9783935546875
1264.1795654296875
1244.1871337890625
1213.3505859375
1188.0186767578125
1172.0203857421875
1121.015380859375
1104.537353515625
1048.2293701171875
1018.4210815429688
992.861083984375
920.1152954101562
884.8330078125
771.6244506835938
775.230224609375
727.4270629882812
688.8736572265625
632.2235717773438
629.0806274414062
531.0064697265625
550.0087890625
555.9593505859375
529.6122436523438
433.5484619140625
477.2158508300781
514.5740966796875
490.2288513183594
398.23468017578125
462.8357849121094
435.83447265625
482.4391174316406
462.0149230957031
432.6463623046875
506.18426513671875
452.42364501953125
425.5073547363281
454.405029296875
30

In [47]:
model = LeNet5(28, 28, 1, 10).cuda()
train_dataset_loader = DataLoader(
    dataset=torchvision.datasets.MNIST(
        root='../data/raw-data/',
        train=True,
        transform=torchvision.transforms.ToTensor()),
    batch_size=600,
    shuffle=False)
test_dataset_loader = DataLoader(
    dataset=torchvision.datasets.mnist.MNIST(
        root='../data/raw-data/',
        transform=torchvision.transforms.ToTensor()),
    batch_size=32,
    shuffle=False)

opti = torch.optim.Adam(model.parameters())
criterion = torch.nn.CrossEntropyLoss(reduction='sum')
loss_sum = 0
for data, target in train_dataset_loader:
    opti.zero_grad()
    output = model(data.cuda())
    loss = criterion(output, target.cuda())
    # print(f'loss_sum = {loss_sum:<20}', end=' | ')
    loss.backward()
    loss_sum += loss.item()
    # print(f'loss = {loss.item():<20}')
    opti.step()

loss_sum = 0                    | loss = 1383.9205322265625  
loss_sum = 1383.9205322265625   | loss = 1382.919677734375   
loss_sum = 2766.8402099609375   | loss = 1379.322021484375   
loss_sum = 4146.1622314453125   | loss = 1376.96630859375    
loss_sum = 5523.1285400390625   | loss = 1375.27294921875    
loss_sum = 6898.4014892578125   | loss = 1374.360595703125   
loss_sum = 8272.762084960938    | loss = 1372.239501953125   
loss_sum = 9645.001586914062    | loss = 1366.7232666015625  
loss_sum = 11011.724853515625   | loss = 1363.2886962890625  
loss_sum = 12375.013549804688   | loss = 1359.122314453125   
loss_sum = 13734.135864257812   | loss = 1351.009521484375   
loss_sum = 15085.145385742188   | loss = 1349.2044677734375  
loss_sum = 16434.349853515625   | loss = 1347.651123046875   
loss_sum = 17782.0009765625     | loss = 1324.5174560546875  
loss_sum = 19106.518432617188   | loss = 1313.9215087890625  
loss_sum = 20420.43994140625    | loss = 1303.45556640625    
loss_sum

In [81]:
def eval_model(model, dataset, device):
    '''
    评估模型
    '''
    model_copy = deepcopy(model).eval()
    correct = 0
    data_loader = DataLoader(dataset, batch_size=32)
    for images, targets in data_loader:
        outputs = model_copy(images.to(device))
        _, predicted = torch.max(outputs, 1)
        correct += torch.eq(predicted, targets.to(device)).sum()
    # print(f'Test Accuracy: {100 * correct / total:.2f}%')
    accuracy = correct / len(dataset)
    return accuracy

eval_model(model, test_dataset, 'cuda')

tensor(0.8580, device='cuda:0')

In [35]:
model_copy = deepcopy(model).eval()
model_copy.eval()
model_copy.cuda()
correct = 0
total = 0
data_loader = test_dataset_loader
for images, targets in data_loader:
    outputs = model_copy(images.cuda())
    _, predicted = torch.max(outputs.data, 1)
    total += targets.size(0)
    correct += (predicted == targets.cuda()).sum().item()
    break
print('Test Accuracy: {:.2f}%'.format(100 * correct / total))

Test Accuracy: 81.25%


In [79]:
for i in range(8, 13):
    print(f'{i:<10}|')
    print(f'{i:.2f}|')

SyntaxError: f-string: single '}' is not allowed (3205282839.py, line 2)